In [ ]:
import gentrl
import rdkit
import torch
import pandas as pd

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools

# Import SDF file

In [ ]:
A1 = PandasTools.LoadSDF("./SDF_name.sdf")

# Convert to Canonical Smiles and exclude unreasonable Smiles

In [ ]:
smi = A1['Smiles']

my_smi = []
for i in smi:
    try:
        mol = Chem.CanonSmiles(i) 
        my_smi.append(mol)
        
    except:
        print("Invalid SMILES:", i)

df = pd.DataFrame(my_smi, columns = ['Smiles'])
df.to_csv('All_Smiles.csv', index = None)

# Create generative tensorial reinforcement learning (GENTRL) models 

In [ ]:
#Default Reward function

from moses.metrics import mol_passes_filters, QED, SA, logP
from moses.metrics.utils import get_n_rings, get_mol

def get_num_rings_6(mol):
    r = mol.GetRingInfo()
    return len([x for x in r.AtomRings() if len(x) > 6])

def penalized_logP(mol_or_smiles, masked = False, default = -5):
    mol = get_mol(mol_or_smiles)
    if mol is None:
        return default
    reward = logP(mol) - SA(mol) - get_num_rings_6(mol)
    if masked and not mol_passes_filters(mol):
        return default
    return reward

#My Reward function

from rdkit.Chem import rdMolDescriptors, Descriptors, Lipinski
from moses.metrics import mol_passes_filters

def lipinski_pass(smiles, masked = False, default = 0):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return default
        
    mol_weight = Descriptors.MolWt(mol)                   #Molecular weight < 500 daltons
    num_hdonors = Lipinski.NumHDonors(mol)                #Hydrogen bond donors <= 5
    num_hacceptors = Lipinski.NumHAcceptors(mol)          #Hydrogen bond acceptors <= 10
    mol_logp = Descriptors.MolLogP(mol)                   #logP < 5
    num_rob = rdMolDescriptors.CalcNumRotatableBonds(mol) #rotatable bonds < 10
        
    if mol_weight >= 500 and num_hdonors > 5 and num_hacceptors > 10 and mol_logp >= 5 and num_rob >= 10:
        reward = 0
    else:
        reward = 1
    if masked and not mol_passes_filters(mol):
        return default
    return reward

In [ ]:
df['pass'] = df['Smiles'].apply(lipinski_pass)
df['plogP'] = df['Smiles'].apply(penalized_logP)
df.to_csv('All_Smiles_lipinski_pass.csv', index = None)

In [ ]:
enc = gentrl.RNNEncoder(latent_size = 50)
dec = gentrl.DilConvDecoder(latent_input_size = 50)
model = gentrl.GENTRL(enc, dec, 50 * [('c', 20)], [('c', 20)], beta = 0.001)
model.cuda();

In [ ]:
md = gentrl.MolecularDataset(sources = [
    {'path': 'All_Smiles_lipinski_pass.csv',
     'smiles': 'Smiles',
     'prob': 1,
     'pass': 'pass',
     'plogP': 'plogP',
    }], 
    props=['pass'])

from torch.utils.data import DataLoader
train_loader = DataLoader(md, batch_size = 50, shuffle = True, num_workers = 1, drop_last = True)

In [ ]:
model.train_as_vaelp(train_loader, lr = 1e-4)

os.mkdir('lipinski_pass_gentrl')
model.save('./lipinski_pass_gentrl/')

# Reinforcement learning

In [ ]:
model.load('lipinski_pass_gentrl/')
model.cuda();

In [ ]:
model.train_as_rl(lipinski_pass)

os.mkdir('lipinski_pass_gentrl_after_rl')
model.save('./lipinski_pass_gentrl_after_rl/')

# Generate new small molecule compounds

In [ ]:
model.load('lipinski_pass_gentrl_after_rl/')
model.cuda();

In [ ]:
generated = []
TrueSmiles = []
while len(TrueSmiles) < 6000:
    sampled = model.sample(1)
    sampled_valid = [s for s in sampled if get_mol(s)]
    generated += sampled_valid
    TrueSmiles = list(set(generated))

In [ ]:
f = open('NEW_Smiles.txt', 'w')
f.close()

for i in TrueSmiles:
    print(i, file = open('NEW_Smiles.txt', 'a'))

# Drawing New Small Molecule Compounds

In [ ]:
mols = []
for smi in TrueSmiles:
    mol = Chem.MolFromSmiles(smi)
    mols.append(mol)
    
Draw.MolsToGridImage(mols, molsPerRow = 3, subImgSize = (300, 200), maxMols = 100,
                     legends = [i for i in TrueSmiles])